# AGENTES FINANCIEROS - "The Club Benefits Programme" FIFA

Los agentes financieros dentro de nuestro proyecto tendran la función de entregar el monto acordado a los diferentes Clubes instalados en los diferentes paises.

# Construcción del conjunto de datos

Para codificar, comience importando las siguientes bibliotecas:

In [114]:
!pip install Faker
import pandas as pd
import uuid
import random
from faker import Faker
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Size**

El tamaño del conjunto de datos será de 5 000. Asigné esta cantidad a una variable constante, que usé en todo momento:

In [115]:
num_agents = 5000

**Características**

Elegí 8 características que dispondra el agente para interactuar con los clubes. Estas características y los respectivos tipos de datos son:

     - ID: una cadena única de caracteres para identificar a cada usuario.
     - Gender (Sexo:) tipo de datos de cadena de tres opciones.
     - Subscriber (Suscriptor:) una opción binaria Verdadero/Falso de su estado de disponibilidad.
     - Name (Nombre:) tipo de datos de cadena del nombre y apellido del usuario.
     - Email (Correo electrónico:) tipo de datos de cadena de la dirección de correo electrónico del usuario.
     - Country (País:) tipo de dato cadena, el país donde se encuentra el agente.
     - bank_country (País del proveedor del banco:) Tipo cadena, genere el código de país ISO 3166-1 alfa-2 del proveedor del banco.
     - bank_Acc_Num (Número de cuenta del banco:) tipo cadena, serie de número de cuenta del banco del agente.

Ingresé lo anterior como una lista de características para inicializar un marco de datos de Pandas:

In [116]:
# A list of 10 features
features = [
    "id",
    "gender",
    "subscriber",
    "name",
    "email",
    "country",
    "bank_country",
    "bank_Acc_Num"
]# Creating a DF for these features
df = pd.DataFrame(columns=features)

**Creación de datos**

**IDs**

Para el atributo ID, utilicé la biblioteca uuid para generar una cadena aleatoria de caracteres 5000 veces. Luego, lo asigné al atributo ID en el marco de datos.

In [117]:
df['id'] = [uuid.uuid4().hex for i in range(num_agents)]

**UUID es una gran biblioteca para generar identificaciones únicas para cada usuario debido a su posibilidad astronómicamente baja de duplicar una identificación.** Es una gran opción cuando se trata de generar conjuntos de caracteres de identificación únicos. Pero, si desea asegurarse de que no se repitieron las ID, puede realizar una verificación simple en el marco de datos con lo siguiente:

In [118]:
print(df['id'].nunique()==num_agents)

True


Esto devolverá True si todas las ID en el conjunto de datos son únicas.

**Gender**

**Este atributo es uno de los casos en los que probablemente no se debería utilizar una elección igualmente aleatoria. Porque se puede suponer con seguridad que cada elección no tiene la misma probabilidad de ocurrir.**

Para el género, proporcioné tres opciones: masculino, femenino y na.

In [119]:
genders = ["Hombre", "Mujer", "No Binario"]

In [120]:
df['gender'] = random.choices(
    genders, 
    weights=(47,47,6), 
    k=num_agents
)

**Subscriber**

Se asigno una selección randómica para determinar que agentes estan ocupados o disponibles.

In [121]:
choice = [True, False]
df['subscriber'] = random.choices(
    choice, 
    k=num_agents
)

**Name**

Aquí usé la biblioteca Faker para crear miles de nombres para todos estos usuarios. **La biblioteca Faker es excelente en esta situación porque tiene una opción para nombres masculinos y femeninos.**}} Para procesar los nombres de género, creé una función para asignar nombres en función de un género determinado.

In [122]:
# Instantiating faker
faker = Faker()

def name_gen(gender):
    """
    Quickly generates a name based on gender
    """
    if gender=='Hombre':
        return faker.name_male()
    elif gender=='Mujer':
        return faker.name_female()
    
    return faker.name()# Generating names for each user
df['name'] = [name_gen(i) for i in df['gender']]

**Email**

Se creo una nueva función que daría formato a los nombres en direcciones de correo electrónico con un nombre de dominio predeterminado. La asignación del dominio sería exclusivo de la FIFA. También manejaría direcciones duplicadas simplemente agregando un número aleatorio al final del nombre formateado:

In [123]:
def emailGen(name, duplicateFound=False):
    """
    Generates a random email address based on the given name. 
    Adds a number at the end if a duplicate address was found.
    """
    # Fake domain name to use
    dom = "@fifa.org"
    
    # Lowercasing and splitting
    name = name.lower().split(" ")
    
    # Random character to insert in the name
    chars = [".", "_"]
    
    new_name = name[0] + random.choice(chars) + name[1] 
    
    # Further distinguishing the email if a duplicate was found
    if duplicateFound:
        
        # Random number to insert at the end
        num = random.randint(0,100)
        
        # Inserting at the end
        new_name = new_name + str(num)
        
    # Returning the email address with the domain name attached
    return new_name + dom

Ahora, para aprovechar adecuadamente el propósito de esta función, creé un ciclo que volvería a ejecutar la función cuando fuera necesario mientras iteraba a través del atributo "Nombre". El ciclo seguiría volviendo a ejecutar la función hasta que se creara un nombre de correo electrónico único.

In [124]:
emails = []

for name in df['name']:
    
    # Generating the email
    email = emailGen(name)
    
    # Looping until a unique email is generated
    while email in emails:
        
        # Creating an email with a random number
        email = emailGen(name, duplicateFound=True)
    
    # Attaching the new email to the list
    emails.append(email)
    
df['email'] = emails

Después de generar todos los correos electrónicos, los asigné al atributo "Correo electrónico" del marco de datos. También puede hacer una verificación opcional para ver que cada correo electrónico sea único con el mismo método que las ID.

**Country**

Se asigno a la columna country los 5000 datos del tamaño del arreglo.

In [125]:
faker = Faker()
df['country'] = [faker.country() for i in range(num_agents)]

**Bank_Country**

Con la función bank_country de faker se asigno una a cada agente dependiendo del país en el que se encuentre.

In [126]:
faker = Faker()

def bc_gen(country):
    """
    Quickly generates a name based on gender
    """
    return faker.bank_country()# Generating country for each user
df['bank_country'] = [bc_gen(i) for i in df['country']]

**Bank_Acc_Num**

Por el comando faker.iban() generamos números de cuentas de bancos internacionales para que estos dispongan de los fondos de la FIFA.

In [127]:
faker = Faker()
df['bank_Acc_Num'] = [faker.iban() for i in range(num_agents)]

**Saving the Dataset**

Una vez generado los datos, los guardamos en un archivo csv para verlos representados.

In [128]:
df.to_csv('dataset_agents.csv')